In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
from torch import nn, optim
import os
from torchvision.models import efficientnet_b1,EfficientNet_B1_Weights, resnet18


In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}


In [3]:
data_dir = 'animal_face'
BATCH_SIZE=80
image_datasets = {x: torchvision.datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


In [4]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_features, in_features, kernel_size=3, padding=1, bias=False)
        self.norm1 = nn.InstanceNorm2d(in_features)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_features, in_features, kernel_size=3, padding=1, bias=False)
        self.norm2 = nn.InstanceNorm2d(in_features)

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.norm1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.norm2(out)
        return identity + out

class DownsampleBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(DownsampleBlock, self).__init__()
        self.conv = nn.Conv2d(in_features, out_features, kernel_size=3, stride=2, padding=1, bias=False)
        self.norm = nn.InstanceNorm2d(out_features)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.norm(x)
        x = self.relu(x)
        return x

class UpsampleBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(UpsampleBlock, self).__init__()
        self.conv = nn.ConvTranspose2d(in_features, out_features, kernel_size=3, stride=2, padding=1, output_padding=1, bias=False)
        self.norm = nn.InstanceNorm2d(out_features)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.norm(x)
        x = self.relu(x)
        return x

class CustomResNet(nn.Module):
    def __init__(self, input_nc, n_residual_blocks=3,num_classes=7):
        super(CustomResNet, self).__init__()
        self.initial_conv = nn.Sequential(
            nn.Conv2d(input_nc, 64, kernel_size=7, padding=3, bias=False),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.downsample_blocks = nn.ModuleList([
            DownsampleBlock(64, 128),
            DownsampleBlock(128, 256)
        ])
        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(256) for _ in range(n_residual_blocks)]
        )
        self.upsample_blocks = nn.ModuleList([
            UpsampleBlock(256, 128),
            UpsampleBlock(128, 64)
        ])
        self.output_conv = nn.Sequential(
            nn.Conv2d(64, 1, kernel_size=7, padding=3),
            nn.Tanh()
        )
        self.average_pool1 = nn.AvgPool2d(2,2)
        self.average_pool2 = nn.AvgPool2d(2,2)
        
        self.flatten = nn.Flatten()
        self.fc_outpout= nn.Linear(3136,num_classes)

    def forward(self, x):
        x = self.initial_conv(x)
        # print("1",x.shape)
        for down_block in self.downsample_blocks:
            x = down_block(x)
        # print("2",x.shape)
        x = self.residual_blocks(x)
        # print("3",x.shape)
        for up_block in self.upsample_blocks:
            x = up_block(x)
        # print("4",x.shape)
        x = self.output_conv(x)

        x = self.average_pool1(x)
        
        x = self.average_pool2(x)
        x = self.flatten(x)
        x = self.fc_outpout(x)
        return x


In [5]:
# model = efficientnet_b1(weights=EfficientNet_B1_Weights.DEFAULT)

# model = models.resnet18(pretrained=True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, len(class_names))

# import torch.nn as nn
# import torchvision.models as models

# # 加载ResNet模型
# resnet = models.resnet18(pretrained=True)

# # 获取全连接层之前的特征提取部分
# features = nn.Sequential(*list(resnet.children())[:-1])

# # 定义新的全连接层和ReLU激活函数
# num_ftrs = resnet.fc.in_features
# fc_layer = nn.Linear(num_ftrs, 256)
# relu = nn.ReLU(inplace=True)

# # 定义模型结构
# class CustomResNet(nn.Module):
#     def __init__(self, features, fc_layer, relu, num_classes=1000):
#         super(CustomResNet, self).__init__()
#         self.features = features
#         self.fc_layer = fc_layer
#         self.relu = relu
#         self.fc_out = nn.Linear(256, num_classes)  # 输出层

#     def forward(self, x):
#         x = self.features(x)
#         x = x.view(x.size(0), -1)
#         x = self.fc_layer(x)
#         x = self.relu(x)
#         x = self.fc_out(x)
#         return x

# 创建模型实例
model = CustomResNet(input_nc=3, num_classes=7)  # 假设输出维度为7

# 输出修改后的ResNet结构
print(model)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


CustomResNet(
  (initial_conv): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
    (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): ReLU(inplace=True)
  )
  (downsample_blocks): ModuleList(
    (0): DownsampleBlock(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (relu): ReLU(inplace=True)
    )
    (1): DownsampleBlock(
      (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (relu): ReLU(inplace=True)
    )
  )
  (residual_blocks): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (norm1): InstanceNorm2d(256, e

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [7]:
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for i, (inputs, labels) in enumerate(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                if i % 10==0:
                    print("{} {} sample".format(phase, i*BATCH_SIZE)) 
                    
                    
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            

    return model


In [8]:
model = train_model(model, criterion, optimizer, num_epochs=1)



Epoch 0/0
----------
train 0 sample
train 800 sample
train 1600 sample
train 2400 sample
train 3200 sample
train 4000 sample
train 4800 sample
train 5600 sample
train 6400 sample
train 7200 sample
train 8000 sample
train 8800 sample
train 9600 sample
train 10400 sample
train 11200 sample
train 12000 sample
train 12800 sample
train Loss: 1.4052 Acc: 0.4751
val 0 sample
val 800 sample
val Loss: 0.7622 Acc: 0.7752


In [7]:
torch.cuda.empty_cache()

In [9]:
save_model_path='models'
checkpoint_path = os.path.join(save_model_path, "my_animal_rec.pth")
torch.save(model, checkpoint_path)
print("Model saved at %s" % checkpoint_path)

Model saved at models\my_animal_rec.pth


In [9]:
a = torch.load("models\\animal_rec.pth")

In [10]:
a.fc_out = torch.nn.Identity()

In [11]:
a

CustomResNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats